In [1]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
%matplotlib inline

# eVTOL energy consumption code

## Section 1. Introduction

### Section 1a. Variable meanings

**V_cruise** : cruise speed in $\frac{m}{s}$

**R** : radius of the rotor in $m$

**A_rotor** : Rotor disk area in $m^2$

**m** : mass in $kg$

$\mathbf{\sigma}$ : thrust weighted solidity ratio

**Cd_mean** : mean blade drag coeffcient

**Fp** : accounts for increase in blade section velocity with rotor edgewise and axial speed

**k** : induced power factor

$\mathbf{\Omega}$ : rotational velocity of the rotor blades in $\frac{rad}{s}$

**P_max** : total deliverable power in $kW$

$\mathbf{\rho}$ : air density in $\frac{kg}{m^3}$ 

**g** : acceleration due to gravity $\frac{m}{s^2}$

$\mathbf{\phi}$ : Rotor tip-path-plane roll angle in degrees

## Section 2. Givens

Defaults the givens to values specified from Matt and Hadi's code. But allows the user to update them following the prompts.

In [2]:
print("Welcome to the eVTOL energy consumption calculator!")

defaultValuesDict = {'V_cruise':50.41, 'R':4.0, 'A_rotor' : 50.26, 'm':2940,
                     'sigma':0.055, 'Cd_mean':0.0089, 'Fp':0.97, 'k':1.75,
                     'Omega':30.12, 'P_max':494.25,'rho':1.225,'g':9.81, 'phi':0}

currentValuesDict = defaultValuesDict

VarNameList = list(defaultValuesDict.keys())
VarValueList = list(defaultValuesDict.values())

UnitsList = ['m/s', 'm', 'm^2', 'kg', '', '', '', '', 'rad/s', 'kW', 'kg/m^3', 'm/s^2', 'deg']

print("The default given variables are as follows:")

for i in range(len(defaultValuesDict)):
    print(f"{VarNameList[i]} = {VarValueList[i]} {UnitsList[i]}")

print()

choice1 = input("Do you want to keep the current givens? (Y/N)")

while choice1 != 'Y' and choice1 != 'y' and choice1 != 'N' and choice1 != 'n':
    choice1 = input('Please enter either Y or N')

if choice1 == 'Y' or choice1 == 'y':
    print('You have chosen to keep the variables as the default values')

else:
    RepeatVariableChange = True
    while RepeatVariableChange:

        VariableName = input('Please select the variable you wish to update, by providing the name as given:')
        while VariableName not in VarNameList:
            VariableName = input('Please enter the variable of your choice exactly as given above: ')

        DoItAgain = True

        while DoItAgain:
            try: 
                x = float(input(f"You have chosen to update {VariableName}. Please enter a number for the new value: "))
                DoItAgain = False
            except:
                DoItAgain = True

        currentValuesDict[VariableName] = x

        indexOfVariable = VarNameList.index(VariableName)
        print(f"{VariableName} has been updated to {x} {UnitsList[indexOfVariable]}")

        choice2 = input('Do you wish to update another variable? (Y/N)')
        while choice2 != 'Y' and choice2 != 'y' and choice2 != 'N' and choice2 != 'n':
            choice2 = input('Please enter either Y or N')

        if choice2 == 'N' or choice2 == 'n':
            RepeatVariableChange = False
            print()
            print('Here are the values of your given variables:')

            VarValueList = list(currentValuesDict.values())
            for i in range(len(currentValuesDict)):
                print(f"{VarNameList[i]} = {VarValueList[i]} {UnitsList[i]}")
        else:
            RepeatVariableChange = True


#create global variables with the values given from the dictionary for use throughout
locals().update(currentValuesDict)
#you can now print(sigma) for example

Welcome to the eVTOL energy consumption calculator!
The default given variables are as follows:
V_cruise = 50.41 m/s
R = 4.0 m
A_rotor = 50.26 m^2
m = 2940 kg
sigma = 0.055 
Cd_mean = 0.0089 
Fp = 0.97 
k = 1.75 
Omega = 30.12 rad/s
P_max = 494.25 kW
rho = 1.225 kg/m^3
g = 9.81 m/s^2
phi = 0 deg

P_max has been updated to 500.0 kW

Here are the values of your given variables:
V_cruise = 50.41 m/s
R = 4.0 m
A_rotor = 50.26 m^2
m = 2940 kg
sigma = 0.055 
Cd_mean = 0.0089 
Fp = 0.97 
k = 1.75 
Omega = 30.12 rad/s
P_max = 500.0 kW
rho = 1.225 kg/m^3
g = 9.81 m/s^2
phi = 0 deg


## Section 3. Math

In [ ]:
def powerCruise_noWind (F_d_value, alpha_value, levelFlightMode=True):
    '''Function Description:
    This function calculates several flight parameters given input conditions. Assumes constant cruise velocity during the duration of the flight and no wind influences
    
    Arguments: 
    F_d value - float value of distance given in kilometers
    alpha - float value of angle of attack given in degrees
    
    Returns:
    None at this time, just print statements for the output
    '''

    if levelFlightMode:
        Theta = alpha_value #Rotor tip-path-plane pitch angle   ###Theta = Alpha for level flight
    else:
        #update this!!!
        Theta = 10
        #currently haven't solved how Theta would be calculated in non-level flight
        pass


    ## Flight Time

    #Assuming constant V_cruise for the duration of the flight
    Tf = (F_d_value*1000)/V_cruise #Time of flight [seconds]
    print(f"The total time of flight is {round(Tf, 2)} seconds, equivalently, {round(Tf/60, 2)} minutes")

    ## Thrust

    T = m*g/(np.cos(phi*np.pi/180)*np.cos(Theta*np.pi/180)) #Total eVTOL thrust [N]
    T_rotor = T/4

    print(f"The thrust on each rotor is {round(T_rotor/1000,2)} kN")
    print(f"The total thrust required is {round(T/1000, 2)} kN")

    ## Induced Velocity (Vi) and Hover Induced Velocity (Vh)

    Vh = np.sqrt(T_rotor/(2*rho*A_rotor))  #Hover induced velocity

    #Induced Velocity
    coefficients = [1, 2*V_cruise*np.sin(alpha_value*np.pi/180), (V_cruise**2), 0, -Vh**4] #Defining the coefficients of the polynomial

    x = np.roots(coefficients) #Solves for the roots of the polynomial

    #Finding the real positive value that is lower than Vh
    for i in range(len(x)):
        if x[i].real > 0 and x[i] < Vh:
            Vi = x[i].real #Induced velocity

    #Checking the solution for Vi based on Equation 10
    y = (Vh**2)/(np.sqrt((V_cruise*np.cos(alpha_value*np.pi/180))**2+(V_cruise*np.sin(alpha_value*np.pi/180)+Vi)**2))

    if y-Vi < 0.001:
        print("Vi is good")
    else:
        print("***Check Vi***")

    ## Power

    #Induced Power
    P_ind_rot = k*T_rotor*Vi

    #Parasite Power
    P_parasite = T*V_cruise*np.sin(alpha_value*np.pi/180) #[W]

    #Profile Power
    P_prof = (rho*A_rotor*((Omega*R)**3)*sigma*Cd_mean*Fp)/8 #[W]

    #Total power required
    P_required = (P_ind_rot*4+P_parasite+P_prof) #[W]
    print(f"The power required for the flight is {round(P_required/1000, 2)} kW")


    ## Energy (Performance Index)

    E_total = (P_required*Tf)/1E6 #[MJ]
    print(f"The total energy consumption during the flight is {round(E_total, 2)} MJ")

    ## Summary

    print(f'Alpha = {alpha_value}°')
    print(f"Flight Distance = {round(F_d_value)} km")
    print(f"Flight Time = {round(Tf, 2)} seconds, equivalently, {round(Tf/60, 2)} minutes")
    print(f"Total Thurst = {round(T/1000, 2)} kN")
    print(f"Thrust Per Rotor = {round(T_rotor/1000, 2)} kN")
    print()
    print("Power:")
    print(f"Induced Power = {round(P_ind_rot*4/1000, 2)} kW")
    print(f"Parasite Power = {round(P_parasite/1000, 2)} kW")
    print(f"Profile Power = {round(P_prof/1000, 2)} kW")
    print(f"Total Power = {round(P_required/1000, 2)} kW")
    print()
    print("Energy:")
    print(f"Total Energy = {round(E_total, 2)} MJ")

In [ ]:
print('Reminder of the current givens: ')
for i in range(len(currentValuesDict)):
    print(f"{VarNameList[i]} = {VarValueList[i]} {UnitsList[i]}")

print()
print('--------------------------------------------------')
print()

for num, i in enumerate(range(20, 50+15, 15)):
    print(f'Study {num+1}:')
    print(f'The flight distance for this study is {i} km')
    powerCruise_noWind (i, alpha_value = 3)
    print()
    print('--------------------------------------------------')
    print()

Reminder of the current givens: 
V_cruise = 50.41 m/s
R = 4.0 m
A_rotor = 50.26 m^2
m = 2940 kg
sigma = 0.055 
Cd_mean = 0.0089 
Fp = 0.97 
k = 1.75 
Omega = 30.12 rad/s
P_max = 494.25 kW
rho = 1.225 kg/m^3
g = 9.81 m/s^2
phi = 0 deg

--------------------------------------------------

Study 1:
The flight distance for this study is 20 km
The total time of flight is 396.75 seconds, equivalently, 6.61 minutes
The thrust on each rotor is 7.22 kN
The total thrust required is 28.88 kN
Vi is good
The power required for the flight is 141.29 kW
The total energy consumption during the flight is 56.06 MJ
Alpha = 3°
Flight Distance = 20 km
Flight Time = 396.75 seconds, equivalently, 6.61 minutes
Total Thurst = 28.88 kN
Thrust Per Rotor = 7.22 kN

Power:
Induced Power = 58.7 kW
Parasite Power = 76.2 kW
Profile Power = 6.39 kW
Total Power = 141.29 kW

Energy:
Total Energy = 56.06 MJ

--------------------------------------------------

Study 2:
The flight distance for this study is 35 km
The total t

## Section 4. Future expansion

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=ec58d3e4-a06a-430a-9b0d-2e414499b170' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>